In [21]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [91]:
def wrangle(file):
    try:
        #read csv
        df = pd.read_csv(file)

        #remove whitespace in columns
        df.columns = df.columns.str.strip()

        # columns to drop
        cols_to_drop =['Adm5', 'Adm6', 'Geo1', 'econ4', 'acc_code', 'FndSrce1', 'function2', 'econ1', 'econ2', 'econ3', 'Year', 'Adm2', 'Adm3', 'Adm4']
        
        #convert numerical columns to float
        num_cols = ['Estimates', 'Revised', 'Executed']
        for col in num_cols:
            df[col] = df[col].str.replace(',', '', regex=True).str.replace(r'[^\d\.\-]', '', regex=True).replace(['', '-'], np.nan).astype(float)
        
        #split funding source
        df[['acc_code', 'fund_source']] = df['FndSrce1'].str.lower().str.split(n=1, expand=True)

        #clean function 1
        df['function1'] = df['function1'].str.lower().str.strip()
        #drop columns
        df.drop(columns = cols_to_drop, inplace=True)
        
        #drop missing rows
        df.dropna(inplace=True)
        
        #reorder columns
        revised_order = ['Class', 'Adm1', 'Econ0', 'function1','fund_source', 'Estimates', 'Revised', 'Executed']
        df = df[revised_order]
        
        #get ratio
        df['execution_ratio'] = np.where(df['Revised'] == 0, 0, df['Executed'] / df['Revised'])
        #remove outliers
        df = df[df['execution_ratio'] <= 2.0]
        
        #execution status
        df['fully_executed'] = (df['execution_ratio']>= 1).astype(int)
        
       
               
        
        return df
        
    except Exception as e:
        print(f'Encountered {e} error')

    


In [92]:
df = wrangle('kenya-boost-2006-2018.csv')
df.head()

/tmp/ipykernel_2760/2983850895.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Class,Adm1,Econ0,function1,fund_source,Estimates,Revised,Executed,execution_ratio,fully_executed
0,Development,Central,Expenditures,agriculture and rural development,domestic resources,18318600.0,14062700.0,5893860.0,0.419113,0
1,Development,Central,Expenditures,agriculture and rural development,domestic resources,12799450.0,9599587.0,3312808.0,0.345099,0
2,Development,Central,Expenditures,agriculture and rural development,domestic resources,7218400.0,5652150.0,2911400.0,0.515096,0
3,Development,Central,Expenditures,agriculture and rural development,domestic resources,10025827.0,8126744.0,4501504.0,0.553912,0
4,Development,Central,Expenditures,agriculture and rural development,domestic resources,500000.0,1130625.0,95000.0,0.084024,0


In [93]:
df['execution_ratio'].describe()

count    75882.000000
mean         0.840188
std          0.309221
min          0.000000
25%          0.690841
50%          0.939446
75%          1.000000
max          2.000000
Name: execution_ratio, dtype: float64

In [28]:
columns = df.columns
print(columns)
for col in columns:
    print(f'Value counts for {col} column')
    try:
        print(df[col].unique())
    except Exception as e:
        print(f"Error in column '{col}': {e}")
    print('------------------')
    print('******************')

Index(['Class', 'Adm1', 'Econ0', 'function1', 'Estimates', 'Revised',
       'Executed', 'fund_source'],
      dtype='object')
Value counts for Class column
['Development' 'Recurrent']
------------------
******************
Value counts for Adm1 column
['Central' 'Social security' 'Local Authorities' ' Central ']
------------------
******************
Value counts for Econ0 column
['Expenditures' 'Below the line' 'A-in-A']
------------------
******************
Value counts for function1 column
['agriculture and rural development' 'human resource development'
 'debt/pensions' 'governance, justice, law and order'
 'public administration and international relations'
 'environment, water and irrigation' 'trade, tourism and industry'
 'special programmes' 'physical infrastructure'
 'research,innovation and technology' 'national security' 'other']
------------------
******************
Value counts for Estimates column
[1.83186000e+07 1.27994500e+07 7.21840000e+06 ... 6.82732842e+10
 7.32605724

In [29]:
df.columns.str.strip()

Index(['Class', 'Adm1', 'Econ0', 'function1', 'Estimates', 'Revised',
       'Executed', 'fund_source'],
      dtype='object')

In [30]:
if 'Estimates' in df.columns:
    print(df['Estimates'].head())
else:
    print('Estimates column not there')

0    18318600.0
1    12799450.0
2     7218400.0
3    10025827.0
4      500000.0
Name: Estimates, dtype: float64


In [41]:
df.info()

AttributeError: 'NoneType' object has no attribute 'info'

In [32]:
df['function1'] = df['function1'].str.lower().str.strip()


In [78]:
df['execution_ratio'] = df['Revised'] / df['Executed']


,Class,Adm1,Econ0,function1,fund_source,Estimates,Revised,Executed,2.38599152338196,2.897719095099988,...,1.4368740640209658,2.027027027027027,1.8404099560761347,1.764122681044348,3.947268170426065,2.0270238074979154,2.027016008677737,2.5,1.7763157894736843,execution_ratio
0,Development,Central,Expenditures,agriculture and rural development,domestic resources,18318600.0,14062700.0,5893860.0,2.385992,2.897719,...,1.436874,2.027027,1.84041,1.764123,3.947268,2.027024,2.027016,2.5,1.776316,2.385992
1,Development,Central,Expenditures,agriculture and rural development,domestic resources,12799450.0,9599587.0,3312808.0,2.385992,2.897719,...,1.436874,2.027027,1.84041,1.764123,3.947268,2.027024,2.027016,2.5,1.776316,2.897719
2,Development,Central,Expenditures,agriculture and rural development,domestic resources,7218400.0,5652150.0,2911400.0,2.385992,2.897719,...,1.436874,2.027027,1.84041,1.764123,3.947268,2.027024,2.027016,2.5,1.776316,1.941386
3,Development,Central,Expenditures,agriculture and rural development,domestic resources,10025827.0,8126744.0,4501504.0,2.385992,2.897719,...,1.436874,2.027027,1.84041,1.764123,3.947268,2.027024,2.027016,2.5,1.776316,1.805340
4,Development,Central,Expenditures,agriculture and rural development,domestic resources,500000.0,1130625.0,95000.0,2.385992,2.897719,...,1.436874,2.027027,1.84041,1.764123,3.947268,2.027024,2.027016,2.5,1.776316,11.901316
